In [ ]:
pip install imblearn

In [7]:
import numpy as np
import boto3
import pandas as pd
import json
from decimal import Decimal
from io import StringIO
from time import sleep

In [8]:
dataset_labeled = pd.read_csv('HotelReservations.csv').drop(['Booking_ID'], axis=1)
dataset_labeled

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,2,0,1,2,0,224,2017,10,2,0,...,False,False,False,False,False,True,False,False,True,1
1,2,0,2,3,0,5,2018,11,6,0,...,False,False,False,False,False,False,True,False,True,2
2,1,0,2,1,0,1,2018,2,28,0,...,False,False,False,False,False,False,True,True,False,1
3,2,0,0,2,0,211,2018,5,20,0,...,False,False,False,False,False,False,True,True,False,2
4,2,0,1,1,0,48,2018,4,11,0,...,False,False,False,False,False,False,True,True,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,False,False,False,False,False,False,True,False,True,3
36271,2,0,1,3,0,228,2018,10,17,0,...,False,False,False,False,False,False,True,True,False,2
36272,2,0,2,6,0,148,2018,7,1,0,...,False,False,False,False,False,False,True,False,True,2
36273,2,0,0,3,0,63,2018,4,21,0,...,False,False,False,False,False,False,True,True,False,2


In [25]:
dynamodb = boto3.resource('dynamodb', region_name = 'us-east-1')
dynamodbTable = dynamodb.Table('HotelReservationsOriginal')

lastEvaluatedKey = None
items = []

while True:
    if lastEvaluatedKey == None:
        response = dynamodbTable.scan()
    else:
        response = dynamodbTable.scan(
        ExclusiveStartKey=lastEvaluatedKey
    )

    items.extend(response['Items'])
    sleep(2)
    
    if 'LastEvaluatedKey' in response:
        lastEvaluatedKey = response['LastEvaluatedKey']
        print(len(items))
    else:
        break

dataset = json.dumps(items, default = float)
dataset_labeled = pd.read_json(StringIO(dataset)).drop(['Booking_ID'], axis=1)
dataset_labeled

,booking_status_Canceled,booking_status_Not_Canceled,type_of_meal_plan_Meal Plan 1,type_of_meal_plan_Meal Plan 2,type_of_meal_plan_Meal Plan 3,market_segment_type_Corporate,repeated_guest,room_type_reserved_Room_Type 7,no_of_children,market_segment_type_Online,...,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 3,required_car_parking_space,room_type_reserved_Room_Type 4,type_of_meal_plan_Not Selected,market_segment_type_Aviation,arrival_month,no_of_previous_cancellations,lead_time
0,False,True,False,True,False,False,0,False,0,False,...,False,False,False,0,False,False,False,7,0,96
1,True,False,True,False,False,False,0,False,0,True,...,False,False,False,0,True,False,False,10,0,267
2,True,False,False,False,False,False,0,False,0,True,...,False,False,False,0,False,True,False,7,0,142
3,True,False,False,False,False,False,0,False,0,True,...,False,False,False,0,False,True,False,5,0,102
4,True,False,False,True,False,False,0,False,0,False,...,False,False,False,0,False,False,False,5,0,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,True,False,False,False,False,False,0,False,0,True,...,False,False,False,0,False,True,False,3,0,28
36271,False,True,False,False,False,False,0,False,0,False,...,False,False,False,0,False,True,False,11,0,121
36272,True,False,False,True,False,False,0,False,0,False,...,False,False,False,0,False,False,False,10,0,386
36273,False,True,True,False,False,False,0,False,0,False,...,False,False,False,0,False,False,False,6,0,192


Label to first column

In [26]:
dataset_labeled.columns
columns = []

columns.append('label_avg_price_per_room')
for i in dataset_labeled.columns:
    if i != 'label_avg_price_per_room':
        columns.append(i)

dataset_labeled = dataset_labeled[columns]
dataset_labeled

,label_avg_price_per_room,booking_status_Canceled,booking_status_Not_Canceled,type_of_meal_plan_Meal Plan 1,type_of_meal_plan_Meal Plan 2,type_of_meal_plan_Meal Plan 3,market_segment_type_Corporate,repeated_guest,room_type_reserved_Room_Type 7,no_of_children,...,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 3,required_car_parking_space,room_type_reserved_Room_Type 4,type_of_meal_plan_Not Selected,market_segment_type_Aviation,arrival_month,no_of_previous_cancellations,lead_time
0,2,False,True,False,True,False,False,0,False,0,...,False,False,False,0,False,False,False,7,0,96
1,2,True,False,True,False,False,False,0,False,0,...,False,False,False,0,True,False,False,10,0,267
2,2,True,False,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,7,0,142
3,2,True,False,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,5,0,102
4,2,True,False,False,True,False,False,0,False,0,...,False,False,False,0,False,False,False,5,0,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,1,True,False,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,3,0,28
36271,1,False,True,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,11,0,121
36272,3,True,False,False,True,False,False,0,False,0,...,False,False,False,0,False,False,False,10,0,386
36273,2,False,True,True,False,False,False,0,False,0,...,False,False,False,0,False,False,False,6,0,192


In [27]:
# let variables between 0-2 for xgboost use
def xgboost_label(room_price_label):
    if room_price_label == 1:
        return 0
    elif room_price_label == 2:
        return 1
    else:
        return 2

In [28]:
dataset_labeled['label_avg_price_per_room'] = dataset_labeled['label_avg_price_per_room'].apply(lambda x: xgboost_label(x))

In [29]:
dataset_labeled

,label_avg_price_per_room,booking_status_Canceled,booking_status_Not_Canceled,type_of_meal_plan_Meal Plan 1,type_of_meal_plan_Meal Plan 2,type_of_meal_plan_Meal Plan 3,market_segment_type_Corporate,repeated_guest,room_type_reserved_Room_Type 7,no_of_children,...,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 3,required_car_parking_space,room_type_reserved_Room_Type 4,type_of_meal_plan_Not Selected,market_segment_type_Aviation,arrival_month,no_of_previous_cancellations,lead_time
0,1,False,True,False,True,False,False,0,False,0,...,False,False,False,0,False,False,False,7,0,96
1,1,True,False,True,False,False,False,0,False,0,...,False,False,False,0,True,False,False,10,0,267
2,1,True,False,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,7,0,142
3,1,True,False,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,5,0,102
4,1,True,False,False,True,False,False,0,False,0,...,False,False,False,0,False,False,False,5,0,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,0,True,False,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,3,0,28
36271,0,False,True,False,False,False,False,0,False,0,...,False,False,False,0,False,True,False,11,0,121
36272,2,True,False,False,True,False,False,0,False,0,...,False,False,False,0,False,False,False,10,0,386
36273,1,False,True,True,False,False,False,0,False,0,...,False,False,False,0,False,False,False,6,0,192


In [30]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(dataset_labeled, test_size=0.3, random_state=40)

In [31]:
df_train.shape

(25392, 32)

In [32]:
df_test.shape

(10883, 32)

In [33]:
X_test = df_test.iloc[:,1:len(dataset_labeled)].astype('float32').values
X_test

array([[  0.,   1.,   1., ...,  11.,   0.,  92.],
       [  1.,   0.,   0., ...,  11.,   0.,  96.],
       [  0.,   1.,   1., ...,   7.,   0.,  12.],
       ...,
       [  1.,   0.,   1., ...,   9.,   0., 433.],
       [  0.,   1.,   1., ...,   3.,   0.,  40.],
       [  0.,   1.,   0., ...,   4.,   0.,  55.]], dtype=float32)

In [34]:
X_test.shape

(10883, 31)

In [35]:
Y_test = df_test.iloc[:, 0].astype('float32').values
Y_test

array([1., 0., 2., ..., 0., 0., 1.], dtype=float32)

In [36]:
Y_test.shape

(10883,)

In [37]:
df_train.astype('float32').to_csv('hotel-reservations-train-xgboost.csv', header = False, index = False)
df_test.astype('float32').to_csv('hotel-reservations-test-xgboost.csv', header = False, index = False)

In [38]:
import collections

collections.Counter(df_train['label_avg_price_per_room'])

Counter({1: 9536, 2: 8198, 0: 7658})

In [39]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

smote = SMOTE()

In [40]:
X_resampled, y_X_resampled = smote.fit_resample(df_train.iloc[:,1:len(dataset_labeled)].astype('float32').values, df_train.iloc[:, 0].astype('float32').values)

In [41]:
collections.Counter(y_X_resampled)

Counter({1.0: 9536, 0.0: 9536, 2.0: 9536})

In [42]:
t = pd.DataFrame(X_resampled)
t.insert(0, column=1000, value = y_X_resampled)
t

,1000,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,1.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,5.000000,0.0,24.000000
1,0.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,8.000000,0.0,303.000000
2,1.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,4.000000,0.0,34.000000
3,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.0,12.000000,0.0,56.000000
4,2.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,9.000000,0.0,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28603,2.0,1.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,8.367228,0.0,0.000000
28604,2.0,1.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.265315,0.734685,0.0,0.0,7.265315,0.0,173.469376
28605,2.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,9.000000,0.0,56.000000
28606,2.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.897199,...,0.0,0.948599,0.0,0.000000,0.051401,0.0,0.0,4.000000,0.0,5.948599


In [43]:
t.to_csv('hotel-reservations-train-xgboost.csv', header = False, index = False)

### Sagemaker configs


In [44]:
import sagemaker
import boto3
from sagemaker import Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [ ]:
# set default credentials
boto3.setup_default_session(region_name='us-east-1')

# create session
session = sagemaker.Session()

# define bucket, folders and keys
bucket = ''
subfolder_model = 'models/hotelreservations/xgboost-oversampling'
subfolder_dataset = 'dataset/hotelreservations'
key_train = 'hotel-train-data-xgboost'
key_test = 'hotel-test-data-xgboost'

role = sagemaker.get_execution_role()

# paths for train and test
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subfolder_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subfolder_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subfolder_model)

In [46]:
import os
with open('hotel-reservations-train-xgboost.csv', 'rb') as f:
  boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subfolder_dataset, 'train', key_train)).upload_fileobj(f)

In [47]:
with open('hotel-reservations-test-xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subfolder_dataset, 'test', key_test)).upload_fileobj(f)

### Treinamento do XGBoost

In [48]:
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

In [49]:
xgboost = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role,
    instance_count = 1,
    instance_type = 'ml.m5.2xlarge',
    output_path = output_location,
    sagemaker_session = session
)

In [50]:
xgboost.set_hyperparameters(
    num_round=200,
    objective='multi:softprob',
    num_class=3
)

In [51]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
data_channels = {'train': train_input}

In [52]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2024-05-26-16-23-54-394


2024-05-26 16:23:54 Starting - Starting the training job...
2024-05-26 16:24:10 Starting - Preparing the instances for training...
2024-05-26 16:24:41 Downloading - Downloading input data...
2024-05-26 16:24:56 Downloading - Downloading the training image...
2024-05-26 16:25:31 Training - Training image download completed. Training in progress..Arguments: train
[2024-05-26:16:25:49:INFO] Running standalone xgboost training.
[2024-05-26:16:25:49:INFO] Path /opt/ml/input/data/validation does not exist!
[2024-05-26:16:25:49:INFO] File size need to be processed in the node: 5.64mb. Available memory size in the node: 23881.41mb
[2024-05-26:16:25:49:INFO] Determined delimiter of CSV input is ','
[16:25:49] S3DistributionType set as FullyReplicated
[16:25:49] 57027x31 matrix with 1313133 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[16:25:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[16:25:49] s

### Deploy

In [ ]:
xgboost_regressor = xgboost.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [54]:
from sagemaker.serializers import CSVSerializer
xgboost_regressor.serializer = CSVSerializer()

In [55]:
X_test.shape, type(X_test)

((10883, 31), numpy.ndarray)

In [56]:
predict_bytes = xgboost_regressor.predict(X_test)
predict_str = predict_bytes.decode('utf-8')
predict_list = eval(predict_str)
predict_array = np.array(predict_list)
predict_array

array([[0.33979878, 0.59169924, 0.068502  ],
       [0.91132796, 0.08744302, 0.00122906],
       [0.00731404, 0.19648093, 0.79620504],
       ...,
       [0.99039781, 0.00559937, 0.00400287],
       [0.85035789, 0.14500892, 0.00463315],
       [0.01885021, 0.96171397, 0.01943576]])

In [57]:
predicted_labels = np.argmax(predict_array, axis=1)
predicted_labels

array([1, 0, 2, ..., 0, 0, 1])

In [58]:
Y_test = np.array(Y_test)
Y_test

array([1., 0., 2., ..., 0., 0., 1.], dtype=float32)

In [59]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

accuracy = accuracy_score(Y_test, predicted_labels)
precision = precision_score(Y_test, predicted_labels, average='macro')
recall = recall_score(Y_test, predicted_labels, average='macro')
f1 = f1_score(Y_test, predicted_labels, average='macro') 

In [60]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.8441606174767987
Precision: 0.8463859642693471
Recall: 0.8463194993373215
F1 Score: 0.8462797189176549


In [61]:
print(classification_report(Y_test, predicted_labels))

              precision    recall  f1-score   support

         0.0       0.86      0.85      0.86      3293
         1.0       0.81      0.80      0.80      4064
         2.0       0.87      0.89      0.88      3526

    accuracy                           0.84     10883
   macro avg       0.85      0.85      0.85     10883
weighted avg       0.84      0.84      0.84     10883



In [62]:
cm = confusion_matrix(Y_test, predicted_labels)
cm

array([[2787,  440,   66],
       [ 395, 3262,  407],
       [  44,  344, 3138]])